<a href="https://colab.research.google.com/github/phanindraarun911/PROJECTS-99/blob/main/Movie_recommendation_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity

In [ ]:
!pip install nltk

import nltk
nltk.download('stopwords')
nltk.download('wordnet')


In [ ]:

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
import string
from requests import get

#Library for Collaborative filtering
!pip install surprise
from surprise import Reader,Dataset,SVD,evaluate
import warnings;warnings.simplefilter('ignore')
%matplotlib inline

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
movies_metadata=pd.read_csv('movies_metadata.csv',error_bad_lines=False)
movies_metadata.head()

In [ ]:
movies_metadata.columns

In [ ]:
movies_metadata.info()

# Lets explore the movies metadata

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(12,9)
sns.heatmap(movies_metadata.isnull(),yticklabels=False,cmap='viridis',ax=ax)

In [ ]:
#From the graph we can easily visualize how much daa is missing for our dataset ,We have lots of data is missing in tagline ,belongs to collection

In [ ]:
#Reading the movie from our small movies data set
movies_small=pd.read_csv('movies.csv')
movies_small.head()

In [ ]:
movies_small.info()

In [ ]:
#Cheking null values in dataset
movies_small.isnull().sum()

In [ ]:
# Creating a plot for Genre Distribution
df1 = movies_small['genres'].apply(lambda genrelist : str(genrelist).split("|"))
df1 = pd.Series(df1).apply(frozenset).to_frame(name='givengenres')
for givengenres in frozenset.union(*df1.givengenres):
    df1[givengenres] = df1.apply(lambda _: int(givengenres in _.givengenres), axis=1)
df1.drop('givengenres',axis=1,inplace=True)
df1['movieId']=movies_small['movieId']
df1 = pd.merge(movies_small,df1,on='movieId')
df1.head()
genre_columns= ['Film-Noir',
       'Romance', 'Western', 'Documentary', 'Thriller', 'Action', 'Musical',
       'War', 'Drama', 'IMAX', 'Crime', 'Children', 'Adventure', 'Horror',
       'Fantasy', 'Animation', 'Comedy', 'Mystery', '(no genres listed)',
       'Sci-Fi']
df1[genre_columns].sum().sort_values(ascending=False).plot(kind='bar',figsize=(12,9))

We can see from the above graph that majority of the movies are of Drama , comedy and thriller


### Plotting the graph to see the distribution of votes across

In [ ]:

#plt.figure(figsize=


most of the vote counts is between 0-5000

In [ ]:
# Look into the distribution of vote average for out movies dataset
plt.figure(figsize=(12,6))
movies_metadata['vote_average'].plot(kind='hist')
plt.xlabel('Vote average')
plt.ylabel('Number of movies')
plt.show()


This is IMDB rating here we can visualize that most of the vote is between 5 to 7

In [ ]:
#Let's create a join plot to see the voete counts and vote average distribution
sns.jointplot(x='vote_average',y='vote_count',data=movies_metadata,alpha=0.7)

From here we can refer one thing that movie whcih have higher rating has higher vote count means more people watch and rate popular movies

In [ ]:
links=pd.read_csv('links.csv')
links.head()

In [ ]:
links.info()

In [ ]:
#Checking Null values
links.isnull().sum()

In [ ]:
movies_metadata.id.isnull().sum()

In [ ]:
links.tmdbId.isnull().sum()

In [ ]:
#Since we are using movies small dataset, we will only keep values in movies_metadata for movies in movies_small
movies_metadata = movies_metadata[movies_metadata.id.isin(links['tmdbId'].astype(str).apply(lambda x:x[:-2]).tolist())]

In [ ]:
#Lets look at the null values after we have created this smaller version of movies_metadata
fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(11.7, 8.27)
sns.heatmap(movies_metadata.isnull(),yticklabels=False,cbar=False,cmap='viridis',ax=ax)

In [ ]:
movies_metadata['genres'] = movies_metadata['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

We also need to add movie id to our movies metadata, we can do it through links.csv

In [ ]:
def convert_int(x):
    try:
        return int(x)
    except:
        return 0

In [ ]:
links['tmdbId'] = links['tmdbId'].apply(convert_int)

In [ ]:
movies_metadata['id'] = movies_metadata['id'].apply(convert_int)

In [ ]:
def return_movieId(tmdbId):
    return links[links['tmdbId']==tmdbId]['movieId'].iloc[0]

In [ ]:
#Get movie Id to the movies_metadata
movies_metadata['movieId'] = movies_metadata['id'].apply(return_movieId)
movies_metadata.head()

In [ ]:
movies_metadata.info()


Since some movies may have low vote average but more number of votes, while other movies may have high vote average and less vote counts, We need a common medium to sort the movies to create top movies chart. For this, let's use IMDB's weighted rating formula to construct top movies chart. Mathematically, it is represented as follows:

Weighted Rating (WR) = (v/(v+m)) R+(m/(v+m)) C

Where,
R = average for the movie (mean) = (Rating)
v = number of votes for the movie = (votes)
m = minimum votes required to be listed in the
C = the mean vote across the whole report

The next step is to determine an appropriate value for m, the minimum votes required to be listed in the chart. We will use 95th percentile as our cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least 95% of the movies in the list.

I will build our overall Top 100 Chart and will define a function to build charts for a particular genre. Let's begin!

<<<<< enter link for IMDB weighted rating here


In [ ]:
no_of_votes = movies_metadata[movies_metadata['vote_count'].notnull()]['vote_count'].astype('int')
vote_mean = movies_metadata[movies_metadata['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_mean.mean()
C

In [ ]:
m = no_of_votes.quantile(0.95)
m

In [ ]:
# Adding year column based on movies release date
movies_metadata['year'] = pd.to_datetime(movies_metadata['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [ ]:
movies_metadata['year'] = pd.to_datetime(movies_metadata['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
top_movies = movies_metadata[(movies_metadata['vote_count'] >= m) & (movies_metadata['vote_count'].notnull()) & (movies_metadata['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
top_movies['vote_count'] = top_movies['vote_count'].astype('int')
top_movies['vote_average'] = top_movies['vote_average'].astype('int')
top_movies.shape

Therefore, to qualify to be considered for the chart, a movie has to have at least 2079 votes on TMDB. We also see that the average rating for a movie on TMDB is 5.916 on a scale of 10. 455 Movies qualify to be on our chart.

In [ ]:
def weighted_rating(x):
  V=x['vote_count']
  R=x['vote_average']
  return (V/(V+m)*R)+(m/(m+V)*C)


In [ ]:
top_movies['wr']=top_movies.apply(weighted_rating,axis=1)

In [ ]:
top_movies = top_movies.sort_values('wr', ascending=False).head(100)

In [ ]:
top_movies.head(15)

In [ ]:
top_movies.columns

In [ ]:
y = movies_metadata.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
y.name = 'genre'
gen_data = movies_metadata.drop('genres', axis=1).join(y)

Let us now construct our function that builds charts for particular genres. For this, we will use relax our default conditions to the 85th percentile instead of 95.

In [ ]:
def top_movies_genre(genre, percentile=0.85):
    df = gen_data[gen_data['genre'] == genre]
    no_of_votes = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_mean = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_mean.mean()
    m = no_of_votes.quantile(percentile)

    top_movies = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    top_movies['vote_count'] = top_movies['vote_count'].astype('int')
    top_movies['vote_average'] = top_movies['vote_average'].astype('int')

    top_movies['wr'] = top_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    top_movies = top_movies.sort_values('wr', ascending=False).head(100)

    return top_movies

In [ ]:
top_movies_genre('Animation').head(10)

## Content Based Recommender
The simple recommender that we just built provides just the top results for the genre, and it shows the the same results for every user looking for that genre.

It also dosen't account for fan following towards particular director or Actors, which accounts for people also watching the movies that are less popular but from famous actors and directors.

For personalized recommendations, We will create a recommendation system that computes similarity between movies based on certain features and recommend movies that are similar to user's taste. As we are using movie's metadata (or content) for creating this system, it is also referred as Content Based Filtering.

We will build four Content Based Recommenders based on:

- Movie overview's that particular user has liked and use latent semantic similarity for comparing similar movies
- Adding Taglines and Movie Overviews and compare using pairwise cosine similarity
- Movie Cast, Crew, Keywords and Genre

#### Movie Description Based Recommender
Let us first try to build a recommender using movie descriptions and taglines. We do not have a quantitative metric to judge our machine's performance so this will have to be done qualitatively.

In [ ]:
movies_metadata.head()

For our first attempt in building Description based recommendation system,
- We will first take list of movies which a user has watched
- Process the description of the movie using NLP techniques like removing stopwords and punctuations, applying Tokenization, lemmatization and stemming, and return a clean list of words
- Using similar techniques, we will process the description/overview of movie in our movies_metadata for the top 80 percentile of movies
- In the next step, we will calculate the similarity between the combined overview of the movies user has watched and the overview of the movies user hasn't watched
- To get this similarity, we will use UMBC's API service to provide latent semantic similarity between 2 scentences. The link to which can be found [here](http://swoogle.umbc.edu/SimService/api.html)

Lets get the top movies that a user has rated more than average

In [ ]:
ratings=pd.read_csv('ratings.csv')
ratings.head()

In [ ]:
#Get movieId for above average ratings for userId 1
ratings[(ratings['userId']==1) & (ratings['rating']>2.5)]['movieId'].tolist()

In [ ]:
def text_process(mess):
    """
    1. remove punc
    2. remove stop words
    3. apply lemmatization
    4. apply stemmization
    5. return list clean overview

    """
    #Remove Stopwords and punctuations
    nopunc = [char for char in mess if char not in string.punctuation]
    stopwords = nltk.corpus.stopwords.words('english')
    nopunc = ''.join(nopunc)

    #Apply tokenization
    tokenized_list = []
    tokenized_list =  [word for word in nopunc.split() if word.lower() not in stopwords]


    wordnet_lemmatizer = WordNetLemmatizer()
    snowball_stemmer = SnowballStemmer('english')

    #Applying Lemmatization

    lemmatized_words = []
    for word in tokenized_list:
        lemmatized_words.append(wordnet_lemmatizer.lemmatize(word))

   #Applying Stemmization

    cleaned_list  = []
    for word in lemmatized_words:
        cleaned_list.append(snowball_stemmer.stem(word))
    return ' '.join(cleaned_list)

In [ ]:

movies_metadata['overview'] = movies_metadata['overview'].astype(str)

In [ ]:
# Pre-processing the overviews for all the movies
movies_metadata['pro_overview'] = movies_metadata['overview'].apply(text_process)

In [ ]:
percentile = 0.90
no_of_votes = movies_metadata[movies_metadata['vote_count'].notnull()]['vote_count'].astype('int')
vote_mean = movies_metadata[movies_metadata['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_mean.mean()
m = no_of_votes.quantile(percentile)

top_movies = movies_metadata[(movies_metadata['vote_count'] >= m) & (movies_metadata['vote_count'].notnull()) & (movies_metadata['vote_average'].notnull())][['movieId','title', 'year', 'vote_count', 'vote_average', 'popularity','pro_overview']]
top_movies['vote_count'] = top_movies['vote_count'].astype('int')
top_movies['vote_average'] = top_movies['vote_average'].astype('int')

In [ ]:
top_movies.sort_values(by='vote_count',ascending=False).head()

In [ ]:
# Using UMBC's API service to get latent sematic similarity score
sss_url = "http://swoogle.umbc.edu/SimService/GetSimilarity"

def sss(s1, s2, type='relation', corpus='webbase'):
    try:
        response = get(sss_url, params={'operation':'api','phrase1':s1,'phrase2':s2,'type':type,'corpus':corpus})
        return float(response.text.strip())
    except:
        #print ('Error in getting similarity for %s: %s' % ((s1,s2), response))
        return 0.0

In [ ]:
user_1_movies=[]
for movieId in ratings[(ratings['userId']==1) & (ratings['rating']>2.5)]['movieId'].tolist():
    user_1_movies.append(movies_metadata[movies_metadata['movieId']==movieId]['pro_overview'].iloc[0])
user_1_movies = ' '.join(user_1_movies)

In [ ]:
user_1_movies

In [ ]:
%%time
top_movies['similarity'] = top_movies['pro_overview'].apply(lambda x:sss(user_1_movies,x))

In [ ]:
top_movies.head()

In [ ]:
top_movies[top_movies.movieId.isin(ratings[ratings['userId']!=1]['movieId'].tolist())][['title','similarity','vote_count','vote_average']].sort_values(by='similarity',ascending=False).head(10)

In [ ]:
#Let's create a recommender based on the above method
def user_taste_recommender(userId,percentile = 0.90):
    no_of_votes = movies_metadata[movies_metadata['vote_count'].notnull()]['vote_count'].astype('int')
    vote_mean = movies_metadata[movies_metadata['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_mean.mean()
    m = no_of_votes.quantile(percentile)

    top_movies = movies_metadata[(movies_metadata['vote_count'] >= m) & (movies_metadata['vote_count'].notnull()) & (movies_metadata['vote_average'].notnull())][['movieId','title', 'year', 'vote_count', 'vote_average', 'popularity','pro_overview']]
    top_movies['vote_count'] = top_movies['vote_count'].astype('int')
    top_movies['vote_average'] = top_movies['vote_average'].astype('int')

    user_movies=[]
    for movieId in ratings[(ratings['userId']==userId) & (ratings['rating']>2.5)]['movieId'].tolist():
        user_movies.append(movies_metadata[movies_metadata['movieId']==movieId]['pro_overview'].iloc[0])
    user_movies = ' '.join(user_movies)

    top_movies['similarity'] = top_movies['pro_overview'].apply(lambda x:sss(user_movies,x))
    top_movies = top_movies[top_movies.movieId.isin(ratings[ratings['userId']!=userId]['movieId'].tolist())][['title','similarity','vote_count','vote_average']].sort_values(by='similarity',ascending=False).head(10)

    return top_movies

In [ ]:
%%time
user_taste_recommender(100)

Just getting recommendation based on movie's synopsis dosent provide eye catching results, and is not reliable enough as Latent Semantic Similarity here takes into account movies from all the genre and most importantly, takes lot of time to calculate through the UMBC's API for soo many movies, So we will try Consine similarity from sklearn's linear kernel which is much faster to calculate

Also, lets add tagline to the description and check if we get better recommendations

In [ ]:
movies_metadata['tagline'] = movies_metadata['tagline'].fillna('')
movies_metadata['description'] = movies_metadata['pro_overview'] + movies_metadata['tagline']
movies_metadata['description'] = movies_metadata['description'].fillna('')

In [ ]:
movies_metadata['description'].head()

In [ ]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies_metadata['description'])

In [ ]:
tfidf_matrix.shape

#### Cosine Similarity
I will be using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two movies. Mathematically, it is defined as follows:

cosine(x,y)=x.y⊺||x||.||y||
Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. Therefore, we will use sklearn's linear_kernel instead of cosine_similarities since it is much faster.

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim[0]

In [ ]:
We now have a pairwise cosine similarity matrix for all the movies in our dataset. The next step is to write a function that returns the 30 most similar movies based on the cosine similarity score.

In [ ]:
movies_metadata = movies_metadata.reset_index()
titles = movies_metadata['title']
indexes = pd.Series(movies_metadata.index, index=movies_metadata['title'])

In [ ]:
#To get pairwise similarity score for movie with index 0
similarity =  list(enumerate(cosine_sim[0]))
print(similarity[:10])

In [ ]:
def desc_based_recommendation(title):
    idx = indexes[title]
    sim = list(enumerate(cosine_sim[idx]))
    #Sorting the list by descending order of similarity
    sim = sorted(sim, key=lambda x: x[1], reverse=True)
    #Taking top 30 similar movies
    sim = sim[1:31]
    rec_movies_indexes = [i[0] for i in sim]
    return titles.iloc[rec_movies_indexes]

Let's check recommendatio for start war

In [ ]:
desc_based_recommendation('Star Wars').head(10)

We get Return of Jedi and Star Wars: The Force Awakens as a recommendation for star wars which is goodl!

#### Metadata Based Recommender

Lets add more details like cast, crew , directors, keywords etc to get better similarity score for movies with similar content.
To do the same we need to prepare this data as our first step.

In [ ]:
#loading data from credits.csv for cast and crew, and Keywords.csv for keywords related to movies
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')

In [ ]:
#Converting id's to int
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
movies_metadata['id'] = movies_metadata['id'].astype('int')

In [ ]:
# Add Cast and Crew column to our movies dataset
movies_metadata = movies_metadata.merge(credits, on='id')
#Add Keywords to the dataset
movies_metadata = movies_metadata.merge(keywords, on='id')

In [ ]:
movies_metadata.head()

After getting the data in a single dataframe, we can get the following from the data:
<p><b> Crew:</b>
Since director is the most important person in the crew of the movie, we will take it as our feature from the crew
<p><b>  Cast: </b>
We will take the first 3 actors from the Cast

In [ ]:
#Checking for Python literal structures: strings, bytes, numbers, tuples, lists, dicts, sets, booleans, and None.
movies_metadata['cast'] = movies_metadata['cast'].apply(literal_eval)
movies_metadata['crew'] = movies_metadata['crew'].apply(literal_eval)
movies_metadata['keywords'] = movies_metadata['keywords'].apply(literal_eval)
#Get the cast and crew size
movies_metadata['cast_size'] = movies_metadata['cast'].apply(lambda x: len(x))
movies_metadata['crew_size'] = movies_metadata['crew'].apply(lambda x: len(x))

In [ ]:
# function to get director from the dict of crew
def get_director(d):
    for i in d:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
movies_metadata['director'] = movies_metadata['crew'].apply(get_director)

In [ ]:
movies_metadata['cast'] = movies_metadata['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
movies_metadata['cast'] = movies_metadata['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [ ]:
movies_metadata['keywords'] = movies_metadata['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

We will add genre, keywords, director and main actors and create count matrix using count vectorizer as we did in Description based recommender and follow similar steps to calculate cosine similarities to get most similar movies.

- Remove Spaces between names
- Convert all features to lower case
- This will help to distinguish between Christopher Nolen and Christopher Columbus
- To get movies with same director more often, we will add director 3 times and provide additional weight to this feature

In [ ]:
#Remove spaces between names
movies_metadata['cast'] = movies_metadata['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
#Remove spaces between names
movies_metadata['director'] = movies_metadata['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
#Add more weight to director
movies_metadata['director'] = movies_metadata['director'].apply(lambda x: [x,x, x])

<b>Keywords:</b>
We only require keywords that occur more than once, having keywords that occur just once will increase complexity and reduce similarity score. So let's count the keywords and keep only those occuring more than once

In [ ]:
k = movies_metadata.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
k.name = 'keyword'

In [ ]:
k = k.value_counts()
k[:5]

In [ ]:
#Removing keyword occuring just once
k = k[k > 1]

Using Snowball Stemmer, lets take the word back to its root form. This helps to reduce same features like forest and forests

In [ ]:
stemmer = SnowballStemmer('english')
stemmer.stem('forests')

In [ ]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in k:
            words.append(i)
    return words

In [ ]:
movies_metadata['keywords'] = movies_metadata['keywords'].apply(filter_keywords)
movies_metadata['keywords'] = movies_metadata['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
movies_metadata['keywords'] = movies_metadata['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
movies_metadata['analyzer'] = movies_metadata['keywords'] + movies_metadata['cast'] + movies_metadata['director'] + movies_metadata['genres']
movies_metadata['analyzer'] = movies_metadata['analyzer'].apply(lambda x: ' '.join(x))

In [ ]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(movies_metadata['analyzer'])

In [ ]:
# Get pairwise cosine similarity
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
movies_metadata = movies_metadata.reset_index()
titles = movies_metadata['title']
indexes = pd.Series(movies_metadata.index, index=movies_metadata['title'])

In [ ]:
desc_based_recommendation('Star Wars').head(10)

We get much better results this time, most of the star war related movies are covered. Let's try for another movie 'Inception'

In [ ]:
desc_based_recommendation('Inception').head(10)

This proves that adding weight to the director definetly works, as most of the movies in Top 10 is of Christopher Nolan

#### Popularity Based Recommendation
Since our current recommender dosen't take popularity and ratings into account, it shows movies like 'Sky Captain and the World of Tomorrow' over many other popular movies.

We will improve our recommendation system by returning only popular movies with more number of ratings

let's take top 25 movies based on similarity scores and calculate the vote of the 70th percentile movie. Then, using this as the value of  m , we will calculate the weighted rating of each movie using IMDB's formula like we did in the Simple Recommender section.

In [ ]:
def popularity_based_recommendations(title,percentile=0.70):
    idx = indexes[title]
    sim = list(enumerate(cosine_sim[idx]))
    sim = sorted(sim, key=lambda x: x[1], reverse=True)
    sim = sim[1:26]
    req_index = [i[0] for i in sim]

    movies = movies_metadata.iloc[req_index][['title', 'vote_count', 'vote_average', 'year']]
    no_of_votes = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_mean = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    m = no_of_votes.quantile(percentile)
    C = vote_mean.mean()
    top_movies = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    top_movies['vote_count'] = top_movies['vote_count'].astype('int')
    top_movies['vote_average'] = top_movies['vote_average'].astype('int')
    top_movies['wr'] = top_movies.apply(weighted_rating, axis=1)
    top_movies = top_movies.sort_values('wr', ascending=False).head(25)
    return top_movies

In [ ]:
popularity_based_recommendations('Star Wars')

We get even better recommendation using popularity based recommender, as we get X-Men and Iron Man 2 in the list , which are my favourites

#### Collaborative Filtering*
The Results from our popularity based recommender are impressive, we get most of the similar movies when querying for a movie. While content based are good when we have good amount of content for the movie like the name of actors, movie synopsis, director’s information etc. we always don’t have all the information required for making relevant recommendations. Also, while we tried to derive user’s taste by using movies overview and taglines as input to our model, the recommendations provided by a collaborative filtering model are way better than a content based model. Another advantage of using a collaborative filtering model over Content based model is that it doesn’t require any data related to movies content. We have built a CF model using Scikit learn’s Surprise library which provides a simple data ingestion for making recommendations through CF. It also provides powerful algorithms like Singular Value Decomposition(SVD) to minimize RMSE and provide great recommendations.

*The code for Collaborative filtering is referred from Rounak Banik's Github Repository which can be accessed [here](https://github.com/rounakbanik/movies/blob/master/movies_recommender.ipynb)

In [ ]:
reader = Reader()

In [ ]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

In [ ]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)

In [ ]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

We get a mean Root Mean Sqaure Error of 0.8951 which is more than good enough for our case. Let us now train on our dataset and arrive at predictions.

In [ ]:
trainset = data.build_full_trainset()
svd.train(trainset)

In [ ]:
#Provide userId, movieId and True Rating
svd.predict(1, 302, 3)

For movie with ID 302, we get an estimated prediction in range of 2.5-3.0. One startling feature of this recommender system is that it doesn't care what the movie is (or what it contains). It works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.

#### Hybrid Recommender*

Hybrid Recommender leverages the best of both Content based and collaborative filtering techniques.

Input: User ID and the Title of a Movie
Output: Similar movies sorted on the basis of expected ratings by that particular user.

*Part of code for Hybrid Recommendation is referred from Rounak Banik's Github Repository which can be accessed from [here](https://github.com/rounakbanik/movies/blob/master/movies_recommender.ipynb)

In [ ]:
links.drop('imdbId',axis=1,inplace=True)
links.columns=['movieId', 'id']
id_map = links.merge(movies_metadata[['title', 'id']], on='id').set_index('title'

In [ ]:
indices_map = id_map.set_index('id')

In [ ]:
def hybrid(userId, title):
    idx = indexes[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']

    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = movies_metadata.iloc[movie_indices][['title', 'vote_count','year', 'id']]
    movies['est rating'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est rating', ascending=False)
    return movies.head(10)

In [ ]:
hybrid(1, 'Avatar')

In [ ]:
hybrid(500, 'Avatar')

We see that for our hybrid recommender, we get different recommendations for different users although the movie is the same. Hence, our recommendations are more personalized and tailored towards particular users

### Conclusion
<p>In this project, I have created 6 types of movie recommender systems:</p>
<p><b>Simple Recommendaion System:</b></p>
- We created Top Movies Charts based on Genre and utilized IMDB's Weighted Rating System to calculate ratings which was used to then sort and return top movies.

<p><b>Content Based Recommendation System:</b> We built four content based recommendation engines</p>
- First we gathered movie's overviews which a user has already seen and rated above average, then we used latent semantic similarity to get the similarity score and created a recommender that provides most similar story to user's liking.
- On our second approach on creating taste based recommendation by using NLP techniques used for above, and added tagline to the description as an input
- Next we considered metadata such as cast, crew, genre and keywords as input features to our Recommendation Engine, We also added weights features like director to get more similar results
- We then improved our prediction by adding a popularity and ratings filter so that recommendations are given on popular movies

<p><b>Collaborative Filtering Recommendation System:</b></p>
- We used the powerful Surprise Library to build a collaborative filter based on single value decomposition(SVD). The RMSE obtained was less than 1 and the engine gave estimated ratings for a given user and movie.

<p><b>Hybrid Recommendation System:</b></p>
- Using ideas from Content based engine and Collaborative filtering based engine, we created a Hybrid recommender system which provided more personalized recommendations for users

The text in the document by Jai Soni is licensed under CC BY 3.0 https://creativecommons.org/licenses/by/3.0/us/

The code in the document by Jai Soni is licensed under the MIT License https://opensource.org/licenses/MIT

## References:

- [1] Kordik, Pavel,https://medium.com/recombee-blog/recommender-systems-explained-d98e8221f468
- [2] Manoj Kumar, D.K. Yadav, Ankur Singh, Vijay Kr. Gupta. International Journal of Computer Applications (0975 – 8887) Volume  124 – No.3, August 2015
- [3] https://en.wikipedia.org/wiki/Recommender_system
- [4] Recommender System for News Articles using Supervised Learning - Scientific Figure on ResearchGate. Available from: https://www.researchgate.net/Collaborative-Vs-Content-Based-Filtering_fig3_318129942 [accessed 21 Apr, 2018]
- [5] Banik, Rounak, https://www.datacamp.com/community/tutorials/recommender-systems-python
- [6] F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. DOI=http://dx.doi.org/10.1145/2827872 , Link for dataset https://grouplens.org/datasets/movielens/
- [7] https://math.stackexchange.com/questions/169032/understanding-the-imdb-weighted-rating-function-for-usage-on-my-own-website
- [8] http://language.worldofcomputing.net/category/tokenization
- [9] https://pythonprogramming.net/stemming-nltk-tutorial/
- [10] http://textminingonline.com/dive-into-nltk-part-iv-stemming-and-lemmatization
- [11] https://en.wikipedia.org/wiki/Tf%E2%80%93idf
- [12] Susan T. Dumais (2005). "Latent Semantic Analysis". Annual Review of Information Science and Technology. 38: 188–230. doi:10.1002/aris.1440380105.
- [13] Lushan Han, Abhay L. Kashyap, Tim Finin, James Mayfield and Johnathan Weese, UMBC_EBIQUITY-CORE: Semantic Textual Similarity Systems, Proc. 2nd Joint Conf. on Lexical and Computational Semantics, Association for Computational Linguistics, June 2013.
- [14]Christian S. Perone http://blog.christianperone.com/2013/09/machine-learning-cosine-similarity-for-vector-space-models-part-iii/
- [15] Banik, Rounak https://github.com/rounakbanik/movies/blob/master/movies_recommender.ipynb
- [16] Image Source: http://bgr.com/2016/03/18/find-movie-forgot-name-using-description/

### How to use this dataset while running the google colab
- upload the datset in files parallel to sample_data(all ready presented in colab)